# PPO Transformer Model



In [31]:
from src.environments.memory import MemoryEnv
from minigrid.wrappers import ViewSizeWrapper
import plotly.express as px 

env = ViewSizeWrapper(MemoryEnv(size = 7, random_length=False, random_start_pos=False, max_steps=200, render_mode='rgb_array'), 7)
obs, info = env.reset()
px.imshow(env.render()).show()
px.imshow(obs['image'][:,:,0].T).show()

In [20]:
obs, _,_,_,_ = env.step(1)
px.imshow(env.render()).show()
px.imshow(obs['image'][:,:,0].T).show()

# Goals: Convert Trajectory Observation Preprocessing in DT training

In [1]:
import sys 
sys.path.append("..")
from src.decision_transformer.offline_dataset import TrajectoryDataset
path = "../trajectories/MiniGrid-MemoryS7FixedStart-v0-Checkpoint11.gz"
dataset = TrajectoryDataset(path, max_len = 10, prob_go_from_end=1)



In [6]:
import pandas as pd 
import plotly.express as px 
probabilities = dataset.get_sampling_probabilities()

pd.DataFrame(dataset.traj_lens[dataset.indices]).describe()


# px.histogram(dataset.traj_lens[dataset.indices]).show()
# px.histogram(probabilities, nbins=100).show()

,0
count,94101.000000
mean,10.201794
std,3.374054
min,2.000000
25%,8.000000
50%,10.000000
75%,12.000000
max,50.000000


In [143]:
from src.decision_transformer.runner import get_max_len_from_model_type

get_max_len_from_model_type("decision_transformer", 13)



10

In [128]:
i = 6836
r = [0] 
while (sum(r) == 0):
    s, a, r, d, rtg, ti, m = dataset[i]
    i = 1 + i
print(i)

6837


In [144]:

i = 0
r = [0] 
positive_rtg_count = 0
for i in range(len(dataset)):
    s, a, r, d, rtg, ti, m = dataset[i]
    i = 1 + i
    if rtg[0] > 0:
        positive_rtg_count += 1

print(positive_rtg_count/i)

38

In [135]:
s, a, r, d, rtg, ti, m = dataset[60002]
show_data(60000).show()

Action: tensor([-10,   4,   6,   2,   4,   2,   2,   2,   1,   2])
RTG: tensor([0.0000, 0.8380, 0.8380, 0.8380, 0.8380, 0.8380, 0.8380, 0.8380, 0.8380,
        0.8380, 0.0000])


In [136]:
from src.visualization import render_minigrid_observations, render_minigrid_observation
import plotly.express as px 


def show_data(i):
    s, a, r, d, rtg, ti, m = dataset[i]
    result = render_minigrid_observations(env, s)
    fig = px.imshow(result, animation_frame=0)
    print(f"Action: {a}")
    print(f"RTG: {rtg.squeeze(-1)}")
    return fig

for i in range(0,10):
    show_data(i).show()


Action: tensor([-10, -10, -10, -10,   2,   2,   2,   2,   0,   2])
RTG: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


Action: tensor([-10,   2,   4,   2,   2,   6,   4,   2,   0,   2])
RTG: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


Action: tensor([-10, -10, -10,   2,   2,   4,   2,   2,   0,   2])
RTG: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


Action: tensor([6, 4, 6, 4, 6, 6, 4, 2, 0, 2])
RTG: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


Action: tensor([-10, -10, -10,   2,   6,   2,   2,   2,   0,   2])
RTG: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


Action: tensor([-10,   2,   4,   2,   2,   6,   4,   2,   0,   2])
RTG: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


Action: tensor([-10, -10, -10, -10,   2,   2,   2,   2,   0,   2])
RTG: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


Action: tensor([-10, -10,   2,   2,   4,   6,   2,   2,   0,   2])
RTG: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


Action: tensor([-10, -10, -10, -10,   2,   2,   2,   2,   0,   2])
RTG: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


Action: tensor([2, 4, 4, 4, 4, 4, 6, 2, 0, 2])
RTG: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [102]:
dataset.rtg

AttributeError: 'TrajectoryDataset' object has no attribute 'rtg'

In [80]:
import torch
torch.concat(dataset.rewards).shape

torch.Size([320000])

In [81]:
dataset.num_trajectories

11552

In [82]:
dataset.num_timesteps

320000

In [83]:
px.histogram([r.max().item() for r in dataset.rewards])

In [84]:
# get an index for all the trajectories which start with a ball on the left.
# get the frame of each trajectory which

In [85]:
px.imshow(render_minigrid_observation(env, dataset.states[0][0]))

In [137]:
import wandb
run = wandb.init()
artifact = run.use_artifact('jbloom/DecisionTransformerInterpretability/MiniGrid-MemoryS7FixedStart-v0__MiniGrid-MemoryS7FixedStart-v0__1__1681336425:v0', type='model')
artifact_dir = artifact.download()

wandb:   1 of 1 files downloaded.  


wandb: Network error (ConnectTimeout), entering retry loop.


In [2]:
import wandb
run = wandb.init()
artifact = run.use_artifact('jbloom/DecisionTransformerInterpretability/MiniGrid-MemoryS7FixedStart-v0__MiniGrid-MemoryS7FixedStart-v0__4__1683706525:v0', type='model')
artifact_dir = artifact.download()

wandb:   1 of 1 files downloaded.  


# Boolean Algebra Examples

1. AND with MLP
2. OR with MLP
3. AND with Attn Head
4. OR with Attn Head

## AND with MLP

In [27]:
import torch 
Win_i = torch.tensor([20, 20])
b = 30

for x in [torch.tensor([0, 0]), torch.tensor([0, 1]), torch.tensor([1, 0]), torch.tensor([1, 1])]:
    a = Win_i @ x - b
    print(f"Give x = {x}, then a = {a} which means {a > 0}")

Give x = tensor([0, 0]), then a = -30 which means False
Give x = tensor([0, 1]), then a = -10 which means False
Give x = tensor([1, 0]), then a = -10 which means False
Give x = tensor([1, 1]), then a = 10 which means True


## OR with MLP

In [35]:
Win_i = torch.tensor([20, 20])
b = 0

for x in [torch.tensor([0, 0]), torch.tensor([0, 1]), torch.tensor([1, 0]), torch.tensor([1, 1])]:
    a = Win_i @ x - b
    print(f"Give x = {x}, then a = {a} which means {a > 0}")

Give x = tensor([0, 0]), then a = 0 which means False
Give x = tensor([0, 1]), then a = 20 which means True
Give x = tensor([1, 0]), then a = 20 which means True
Give x = tensor([1, 1]), then a = 40 which means True


## AND with Attn Head

In [ ]:
import torch 
Win_i = torch.tensor([20, 20])
b = 30

for x in [torch.tensor([0, 0]), torch.tensor([0, 1]), torch.tensor([1, 0]), torch.tensor([1, 1])]:
    a = Win_i @ x - b
    print(f"Give x = {x}, then a = {a} which means {a > 0}")